In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd

In [ ]:
map_name = "testtrack"

checkpoints = []
with open(f"{map_name}.json") as f:
    map_info = json.load(f)
    checkpoints = map_info["checkpoints"]

In [ ]:
def get_distance(x1, y1, x2, y2):
    '''get_distance Returns the distance between two points
    '''
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def calc_current_waypoint(x, y):
    '''get_current_checkpoint Returns the current checkpoint
    '''
    check_dists = []
    for checkpoint in checkpoints:
        check_dists.append(get_distance(x, y, checkpoint[0], checkpoint[1]))
    
    closest_check_dist_idx = int(np.argmin(check_dists))
    closest_idx = int(closest_check_dist_idx)
    dist_to_closest = check_dists[closest_check_dist_idx]
    closest_wpt = checkpoints[closest_idx]
    
    dist_to_next = check_dists[min(closest_check_dist_idx + 1, len(check_dists) - 1)]
    next_wpt = checkpoints[min(closest_idx + 1, len(check_dists) - 1)]

    dist_btw_next = get_distance(closest_wpt[0], closest_wpt[1], next_wpt[0], next_wpt[1])

    if dist_to_closest < 120.0 or (dist_to_next < dist_btw_next + 50.0 and closest_idx < len(checkpoints) - 1):
        closest_idx += 1

    return max(min(closest_idx, len(checkpoints) - 1), 0)

In [ ]:
density = 1000
scaling = 3500.0 / float(density)
wpt_map = np.zeros((density, density))
for x in range(0, density):
    for y in range(density):
        wpt_map[x][y] = calc_current_waypoint(int(x * scaling), int(y * scaling)) % 5
        

In [ ]:
map_name = "testtrack"
## Plotting
fig, ax = plt.subplots()
wpt_map_adj = np.transpose(wpt_map)
ax.imshow(wpt_map_adj, cmap='Set3', interpolation='nearest')

## Overlay waypoints
color_idx = 0
for checkpoint in checkpoints:
    color_idx += 1
    color_idx %= 5
    ax.scatter(checkpoint[0] / scaling, checkpoint[1] / scaling, color='black', s=25)

## Overlay map png on top of heatmap
img = plt.imread(f"assets/{map_name}_clear.png")
ax.imshow(img, alpha=0.8, extent=[0, 1000, 0, 1000])
## set plot size to density
fig.set_size_inches(density / 100, density / 100)

## hide axes
ax.axis('off')

plt.show()

In [ ]:
def calc_course_progress(x,y) -> float:

    percent_per_checkpt = 100.0 / float(len(checkpoints) - 1)
    checkpt_idx = calc_current_waypoint(x,y) - 1

    base_percentage = percent_per_checkpt * checkpt_idx
    distance_to_checkpt = get_distance(x, y, checkpoints[checkpt_idx + 1][0], checkpoints[checkpt_idx + 1][1])
    distance_btw_checkpt = get_distance(checkpoints[checkpt_idx][0], checkpoints[checkpt_idx][1], checkpoints[min(checkpt_idx + 1, len(checkpoints) - 1)][0], checkpoints[min(checkpt_idx + 1, len(checkpoints) - 1)][1]) - 120.0
    percent_to_next = ((distance_btw_checkpt - distance_to_checkpt) / distance_btw_checkpt) * percent_per_checkpt
    return max(min(round(base_percentage + percent_to_next, 3), 99.999), 0.1)

course_length_list = [0.0 for _ in checkpoints]

def get_total_course_length(idx: int = len(checkpoints) - 1) -> float:
    if idx <= 0:
        return 0.0
    if course_length_list[idx] != 0:
        return course_length_list[idx]
    total_length = 0.0
    for i in range(0, idx):
        total_length += get_distance(checkpoints[i][0], checkpoints[i][1], checkpoints[i + 1][0], checkpoints[i + 1][1])
    course_length_list[idx] = total_length
    return total_length

def get_course_progress(x, y) -> float:
    percent_per_unit = 100.0 / get_total_course_length()
    checkpt_idx = calc_current_waypoint(x,y) - 1

    base_percentage = percent_per_unit * get_total_course_length(checkpt_idx)
    distance_to_checkpt = get_distance(x, y, checkpoints[checkpt_idx + 1][0], checkpoints[checkpt_idx + 1][1])
    distance_btw_checkpt = get_distance(checkpoints[checkpt_idx][0], checkpoints[checkpt_idx][1], checkpoints[min(checkpt_idx + 1, len(checkpoints) - 1)][0], checkpoints[min(checkpt_idx + 1, len(checkpoints) - 1)][1]) - 120.0
    percent_to_next = (distance_btw_checkpt - distance_to_checkpt) * percent_per_unit
    return max(min(round(base_percentage + percent_to_next, 3), 99.999), 0.1)

In [ ]:

completion_map = np.zeros((density, density))
for x in range(0, density):
    for y in range(density):
        completion_map[x][y] = get_course_progress(int(x * scaling), int(y * scaling))
        

In [ ]:
map_name = "testtrack"
## Plotting
fig, ax = plt.subplots()
completion_map_adj = np.transpose(completion_map)
ax.imshow(completion_map_adj, cmap='Greys', interpolation='nearest')

## Overlay waypoints
color_idx = 0
for checkpoint in checkpoints:
    color_idx += 1
    color_idx %= 5
    ##ax.scatter(checkpoint[0] / scaling, checkpoint[1] / scaling, color='black', s=25)

## Overlay map png on top of heatmap
img = plt.imread(f"assets/{map_name}_clear.png")
ax.imshow(img, alpha=0.8, extent=[0, 1000, 0, 1000])
## set plot size to density
fig.set_size_inches(density / 100, density / 100)

## hide axes
ax.axis('off')

plt.show()

In [ ]:
map_name = "circuit1_b"
new_start = "R"

In [ ]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
offset = alphabet.index(new_start)
input("CAUTION: This will modify the JSON and XP files for the map. Press enter to continue.")

## Modify JSON checkpoints to start at new start alphabetically
with open(f"{map_name}.json", "r") as f:
    data = json.load(f)
    checkpts = data["checkpoints"]
    checkpts = checkpts[1:-1]
    checkpts = checkpts[offset:] + checkpts[:offset]
    checkpts = [checkpoints[0]] + checkpts + [checkpoints[-1]]
    data["checkpoints"] = checkpts
with open(f"{map_name}.json", "w") as f:
    json.dump(data, f, indent=4)
## Modify checkpoints to start at new start alphabetically

alphabet = alphabet[:len(checkpts) - 3]
with open(f"{map_name}.xp", "r") as f:
    lines = f.readlines()
    start_replace = False
    new_map_lines = []
    for line in lines:
        line = line.strip()
        if not start_replace:
            if line.startswith("mapData:"):
                start_replace = True
            new_map_lines.append(line)
            continue
        if line.startswith("End"):
            start_replace = False
            new_map_lines.append(line)
            continue
        for char in line:
            if char in alphabet:
                start_idx = alphabet.index(char)
                new_idx = (start_idx - offset) % len(alphabet)
                line = line.replace(char, alphabet[new_idx])
        new_map_lines.append(line)
with open(f"{map_name}.xp", "w") as f:
    f.writelines(new_map_lines)
